# Downloading Dataset

In [ ]:
import csv
import glob
import os
import numpy as np
from tqdm import tqdm

In [ ]:
src_directory = "../../dataset/micro/"
files = [f for f in glob.glob(src_directory + "*" + ".mp4", recursive=True)]
already_downloaded = []

for f in files:
    already_downloaded.append(f.replace(src_directory, ""))

    
train_labels_file = "../../dataset/csv_files/train_labels.csv"
with open(train_labels_file, mode='r') as infile:
    reader = csv.reader(infile)
    train_list = {rows[0]: rows[1] for rows in reader}
    infile.close()
train_list.pop("filename", None)


total_stalled = 0
total_nonstall = 0

for data in train_list.keys():
    if train_list[data] == '1':
        total_stalled = total_stalled + 1
    elif train_list[data] == '0':
        total_nonstall = total_nonstall + 1


print("Already available files: " + str(len(already_downloaded)))
print("Files in full dataset: " + str(len(train_list)) + " non stall: " + 
      str(total_nonstall) + " stall: " + str(total_stalled))

In [ ]:
# remove all the files that are already downloaded from the full list

for file in range(len(already_downloaded)):
    train_list.pop(already_downloaded[file], None)
    

unseen_stalled = []
unseen_nonstall = []

for data in train_list.keys():
    if train_list[data] == '1':
        unseen_stalled.append(data)
    elif train_list[data] == '0':
        unseen_nonstall.append(data)

        
print("Unseen non stall: " + str(len(unseen_nonstall)) + " stall: " + str(len(unseen_stalled)))

In [ ]:
train_metadata_file = "../../dataset/csv_files/train_metadata.csv"


with open(train_metadata_file, mode='r') as infile:
    reader = csv.reader(infile)
    metadata = {rows[0]: rows[1] for rows in reader}
    infile.close()
metadata.pop("filename", None)

In [ ]:
download_directory = "../../download/"

new_downloaded = []

In [ ]:
downloadable = len(unseen_stalled)

for i in tqdm(range(downloadable)):
    
    data = unseen_stalled[0]
    url = metadata[data]
    
    os.system("aws s3 cp " + url + " " + download_directory + " --no-sign-request")
    
    new_downloaded.append(data)
    unseen_stalled.remove(data)
   

print("done")

In [ ]:
print(len(new_downloaded))

In [ ]:
for i in range(len(already_downloaded)):
    already_downloaded[i] = already_downloaded[i].replace('../../dataset/micro\\', '')
    already_downloaded[i] = already_downloaded[i].replace('.mp4', '')
    
print(len(already_downloaded))

In [ ]:
new_csvs = ["stall", "nonstall_0", "nonstall_2", "nonstall_3", "nonstall_4", "nonstall_5", "nonstall_6", "nonstall_7"]

all_files = already_downloaded

for i in range(8):
    with open("../Data Downloader/" + new_csvs[i] + ".csv", mode='r') as infile:
        reader = csv.reader(infile)
        files = [rows[0] for rows in reader]
        infile.close()

    all_files = all_files + files
    print(len(files), len(all_files))


In [ ]:
list.sort(all_files)
print(all_files)

In [ ]:
labels = []

for i in range(len(all_files)):
    all_files[i] = all_files[i] + '.mp4'
    labels.append(train_list[all_files[i]])
    


In [ ]:
import pandas as pd

data = list(zip(all_files,labels))

df = pd.DataFrame(data = data)

df.to_csv("mili.csv", index=False, header=False)

In [ ]:
import csv
from sklearn.metrics import matthews_corrcoef as mcc

csv1 = "../../Progress/resnet101/submission3D.csv"
csv2 = "../../Progress/resnet152/submission3D.csv"
csv3 = "../../Progress/resnet200/submission3D.csv"


with open(csv1, mode='r') as infile:
    reader = csv.reader(infile)
    labels1 = [rows[1] for rows in reader]
    infile.close()
labels1.remove(labels1[0])
labels1 = list(map(int, labels1))

with open(csv2, mode='r') as infile:
    reader = csv.reader(infile)
    labels2 = [rows[1] for rows in reader]
    infile.close()
labels2.remove(labels2[0])
labels2 = list(map(int, labels2))


with open(csv3, mode='r') as infile:
    reader = csv.reader(infile)
    labels3 = [rows[1] for rows in reader]
    infile.close()
labels3.remove(labels3[0])
labels3 = list(map(int, labels3))


print(mcc(labels1, labels2))
print(mcc(labels2, labels3))
print(mcc(labels3, labels1))

In [7]:
from math import sqrt
import numpy as np

total_data = 14160
tolerance = 0.0001

#detected_pos_list = [1008, 1026, 1211, 1349, 1021]
#mcc_list = [0.5820, 0.5820, 0.5353, 0.5101, 0.4906]

detected_pos_list = [1008, 1026, 1211, 1349, 1021, 1267, 1206, 1167, 1772, 1762]
mcc_list = [0.5820, 0.5820, 0.5353, 0.5101, 0.4906, 0.6160, 0.6151, 0.4965, 0.4581, 0.3695]

probable_stall = []



for i in range(len(mcc_list)):
    
    stall_here = []
    detected_pos = detected_pos_list[i]
    LB = mcc_list[i]
    
    for stall in range(1,3000):
        for TP in range(1,min(detected_pos, stall)+1):
            
            FP = detected_pos-TP
            FN = stall-TP
            TN = total_data-detected_pos+TP-stall

            MCC = float(TP*TN - FP*FN)/sqrt(float(TP+FP)*float(TP+FN)*float(TN+FP)*float(TN+FN))

            if MCC >= (LB - tolerance) and MCC <= (LB + tolerance):
                stall_here.append(stall)
                
                
    print(len(stall_here))
    probable_stall.append(stall_here)


    
common = np.array(probable_stall[0])

for i in range(1, len(mcc_list)):
    common = np.intersect1d(common, np.array(probable_stall[i]))
    
print(common)

480
530
672
678
624
608
557
657
765
765
[]


[1808]


In [ ]:
print(np.intersect1d(common, d))

In [ ]:
idea = [5,3]
idea2 = [1,4, 3]

In [ ]:
idea3 = []
idea3.append(idea)
idea3.append(idea2)
print(idea3)

In [1]:
import csv
import glob
import os
import numpy as np
from tqdm import tqdm

In [21]:
labels_file = "../../dataset/csv_files/train_labels.csv"


with open(labels_file, mode='r') as infile:
    reader = csv.reader(infile)
    labels = {rows[0]: rows[1] for rows in reader}
    infile.close()
labels.pop("filename", None)

'stalled'

In [9]:
test_split_file = "../../fold_3_test.csv"

with open(test_split_file, mode='r') as infile:
    reader = csv.reader(infile)
    test = [rows[0] for rows in reader]
    infile.close()
test.remove(test[0])



tot_stall = 0
tot_nonstall = 0

for i in range(len(test)):
    if labels[test[i]] == '0':
        tot_nonstall = tot_nonstall + 1
    else:
        tot_stall = tot_stall + 1
        
print(tot_nonstall, tot_stall)

{'F', 'C', 'D', 'M', 'E', 'A', 'H', 'L', 'I', 'G', 'J', 'K'}


In [41]:
src_directory = "../../dataset/new/nonstall_3\\"
files = [f for f in glob.glob(src_directory + "*" + ".mp4", recursive=True)]
non3 = []

for f in files:
    non3.append(f.replace(src_directory, ""))

In [43]:
print(len(test)+ len(non1) + len(non2) + len(non3))

3599


In [44]:
new_test = test + non1 + non2 + non3

In [45]:
len(new_test)

3599

In [50]:
new_test = np.sort(new_test)
print(new_test)

['100083.mp4' '100260.mp4' '100616.mp4' ... '686626.mp4' '687129.mp4'
 '687153.mp4']


In [52]:
new_test_labels = []

for i in range(len(new_test)):
    new_test_labels.append(labels[new_test[i]])

In [57]:
import pandas as pd

data = list(zip(new_test,new_test_labels))

df = pd.DataFrame(data = data)

df.to_csv('fold_3_test.csv', index=False, header=["filename", "class"])

# New Files Distribution and split generation

In [1]:
import csv
import glob
import os
import numpy as np
from tqdm import tqdm
import pandas as pd

### Metadata and Labels load

In [3]:
metadata_file = "../../dataset/csv_files/train_metadata.csv"
labels_file = "../../dataset/csv_files/train_labels.csv"

with open(metadata_file, mode='r') as infile:
    reader = csv.reader(infile)
    exp_ID = {rows[0]:rows[2] for rows in reader}
    infile.close()
exp_ID.pop("filename", None)


with open(labels_file, mode='r') as infile:
    reader = csv.reader(infile)
    labels = {rows[0]: rows[1] for rows in reader}
    infile.close()
labels.pop("filename", None)


print("Done")

Done


### Load current dataset files list

In [4]:
mili_file = "../Data Downloader/mili.csv"

with open(mili_file, mode='r') as infile:
    reader = csv.reader(infile)
    new_dataset_list = [rows[0] for rows in reader]
    infile.close()
    
print("Done")

Done


### Distribution of Experiment IDs in Full set

In [12]:
distribution_list_full = {}

for current_data in exp_ID.keys():
    this_ID = exp_ID[current_data]
    
    if this_ID in distribution_list_full:
        filenames_here = distribution_list_full[this_ID]
        filenames_here.append(current_data)
        distribution_list_full[this_ID] = filenames_here
    
    else:
        distribution_list_full[this_ID] = [current_data]
        
        
unique_IDs = np.sort(list(distribution_list_full.keys()))

print("Unique IDs: ", unique_IDs)

Unique IDs:  ['A' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M']


In [13]:
for i in range(len(unique_IDs)):
    
    exp_id = unique_IDs[i]
    data_here = distribution_list_full[exp_id]
    data_len = len(data_here)
    
    stalls_here = []
    nonstall_here = []
    
    for i in range(data_len):
        if labels[data_here[i]] == '1':
            stalls_here.append(data_here[i])
        else:
            nonstall_here.append(data_here[i])
    
    print(exp_id + "\t" + str(data_len) + "\tstall\t" + str(len(stalls_here)) + "\tnonstall\t" + str(len(nonstall_here)))

A	143495	stall	284	nonstall	143211
C	67023	stall	203	nonstall	66820
D	20454	stall	46	nonstall	20408
E	31832	stall	73	nonstall	31759
F	60794	stall	371	nonstall	60423
G	81994	stall	221	nonstall	81773
H	74231	stall	67	nonstall	74164
I	21851	stall	197	nonstall	21654
J	5037	stall	39	nonstall	4998
K	27077	stall	92	nonstall	26985
L	444	stall	133	nonstall	311
M	38816	stall	161	nonstall	38655


### Distribution of Experiment IDs in current Dataset

In [14]:
distribution_list = {}

for i in range(len(new_dataset_list)):
    current_data = new_dataset_list[i]
    this_ID = exp_ID[current_data]
    
    if this_ID in distribution_list:
        filenames_here = distribution_list[this_ID]
        filenames_here.append(current_data)
        distribution_list[this_ID] = filenames_here
    
    else:
        distribution_list[this_ID] = [current_data]

In [15]:
for i in range(len(unique_IDs)):
    
    exp_id = unique_IDs[i]
    data_here = distribution_list[exp_id]
    data_len = len(data_here)
    
    stalls_here = []
    nonstall_here = []
    
    for i in range(data_len):
        if labels[data_here[i]] == '1':
            stalls_here.append(data_here[i])
        else:
            nonstall_here.append(data_here[i])
    
    print(exp_id + "\t" + str(data_len) + "\tstall\t" + str(len(stalls_here)) + "\tnonstall\t" + str(len(nonstall_here)))

A	2289	stall	284	nonstall	2005
C	1183	stall	203	nonstall	980
D	342	stall	46	nonstall	296
E	510	stall	73	nonstall	437
F	1249	stall	371	nonstall	878
G	1694	stall	221	nonstall	1473
H	1018	stall	67	nonstall	951
I	544	stall	197	nonstall	347
J	188	stall	39	nonstall	149
K	467	stall	92	nonstall	375
L	412	stall	133	nonstall	279
M	684	stall	161	nonstall	523


### Creation of train test split

In [29]:
import random

nonstall_stall_ratio = 90/10

test_stall_per_ID = 10

random.seed(100)

split_no = 0



test_nonstall_per_ID = nonstall_stall_ratio * test_stall_per_ID

# total_test_size = (test_stall_per_ID + test_nonstall_per_ID) * len(unique_IDs)
# total_stalls = test_stall_per_ID * len(unique_IDs)
# total_nonstalls = test_nonstall_per_ID * len(unique_IDs)

train_split_files = []
test_split_files = []


for i in range(len(unique_IDs)):
    
    exp_id = unique_IDs[i]
    data_here = distribution_list[exp_id]
    data_len = len(data_here)
    
    stalls_here = []
    nonstall_here = []
    
    for i in range(data_len):
        if labels[data_here[i]] == '1':
            stalls_here.append(data_here[i])
        else:
            nonstall_here.append(data_here[i])
    
    random.shuffle(stalls_here)
    random.shuffle(nonstall_here)
    
    for i in range(len(stalls_here)):
        if i < test_stall_per_ID :
            test_split_files.append(stalls_here[i])
        else:
            train_split_files.append(stalls_here[i])
            
    for i in range(len(nonstall_here)):
        if i < test_nonstall_per_ID :
            test_split_files.append(nonstall_here[i])
        else:
            train_split_files.append(nonstall_here[i])
   

print("Done! ")

Done! 


In [30]:
# sanity check

if len(list(set(test_split_files).intersection(train_split_files))) > 0:
    print("Error")

if len(test_split_files) == len(set(test_split_files)) and len(train_split_files) == len(set(train_split_files)) :
    print("No Duplicates in each set")
else:
    print("Error")
    
print("Train len: " + str(len(train_split_files)))
print("Test len: " + str(len(test_split_files)))

No Duplicates in each set
Train len: 9380
Test len: 1200


In [32]:
distribution_of_test = {}
print("Printing test distribution: ")

for i in range(len(test_split_files)):
    current_data = test_split_files[i]
    this_ID = exp_ID[current_data]
    
    if this_ID in distribution_of_test:
        filenames_here = distribution_of_test[this_ID]
        filenames_here.append(current_data)
        distribution_of_test[this_ID] = filenames_here
    
    else:
        distribution_of_test[this_ID] = [current_data]
        
        
for i in range(len(unique_IDs)):
    
    exp_id = unique_IDs[i]
    data_here = distribution_of_test[exp_id]
    data_len = len(data_here)
    
    stalls_here = []
    nonstall_here = []
    
    for i in range(data_len):
        if labels[data_here[i]] == '1':
            stalls_here.append(data_here[i])
        else:
            nonstall_here.append(data_here[i])
    
    print(exp_id + "\t" + str(data_len) + "\tstall\t" + str(len(stalls_here)) + "\tnonstall\t" + str(len(nonstall_here)))

Printing test distribution: 
A	100	stall	10	nonstall	90
C	100	stall	10	nonstall	90
D	100	stall	10	nonstall	90
E	100	stall	10	nonstall	90
F	100	stall	10	nonstall	90
G	100	stall	10	nonstall	90
H	100	stall	10	nonstall	90
I	100	stall	10	nonstall	90
J	100	stall	10	nonstall	90
K	100	stall	10	nonstall	90
L	100	stall	10	nonstall	90
M	100	stall	10	nonstall	90


In [33]:
distribution_of_train = {}
print("Printing train distribution: ")

for i in range(len(train_split_files)):
    current_data = train_split_files[i]
    this_ID = exp_ID[current_data]
    
    if this_ID in distribution_of_train:
        filenames_here = distribution_of_train[this_ID]
        filenames_here.append(current_data)
        distribution_of_train[this_ID] = filenames_here
    
    else:
        distribution_of_train[this_ID] = [current_data]
        
        
for i in range(len(unique_IDs)):
    
    exp_id = unique_IDs[i]
    data_here = distribution_of_train[exp_id]
    data_len = len(data_here)
    
    stalls_here = []
    nonstall_here = []
    
    for i in range(data_len):
        if labels[data_here[i]] == '1':
            stalls_here.append(data_here[i])
        else:
            nonstall_here.append(data_here[i])
    
    print(exp_id + "\t" + str(data_len) + "\tstall\t" + str(len(stalls_here)) + "\tnonstall\t" + str(len(nonstall_here)))

Printing train distribution: 
A	2189	stall	274	nonstall	1915
C	1083	stall	193	nonstall	890
D	242	stall	36	nonstall	206
E	410	stall	63	nonstall	347
F	1149	stall	361	nonstall	788
G	1594	stall	211	nonstall	1383
H	918	stall	57	nonstall	861
I	444	stall	187	nonstall	257
J	88	stall	29	nonstall	59
K	367	stall	82	nonstall	285
L	312	stall	123	nonstall	189
M	584	stall	151	nonstall	433


### Write to csv

In [37]:
train_split_files = np.sort(train_split_files)
test_split_files = np.sort(test_split_files)


train_split_labels = []
test_split_labels = []

for i in range(len(train_split_files)):
    train_split_labels.append(labels[train_split_files[i]])

for i in range(len(test_split_files)):
    test_split_labels.append(labels[test_split_files[i]])

In [39]:
df = pd.DataFrame(data = list(zip(train_split_files,train_split_labels)))
df.to_csv("fold_" + str(split_no) + "_train.csv", index=False, header=["filename", "class"])

df = pd.DataFrame(data = list(zip(test_split_files,test_split_labels)))
df.to_csv("fold_" + str(split_no) + "_test.csv", index=False, header=["filename", "class"])

### Just some test

In [7]:
import csv
import glob
import os
import numpy as np
from tqdm import tqdm
import pandas as pd



with open("../../inference_partho.csv", mode='r') as infile:
    reader = csv.reader(infile)
    filenames = [rows[0] for rows in reader]
    infile.close()
filenames.remove(filenames[0])
    
with open("../../inference_partho.csv", mode='r') as infile:
    reader = csv.reader(infile)
    labels_1 = [rows[1] for rows in reader]
    infile.close()
labels_1.remove(labels_1[0])
labels_1 = np.array(labels_1, dtype=np.uint8)

with open("../../inference_sayeed.csv", mode='r') as infile:
    reader = csv.reader(infile)
    labels_2 = [rows[1] for rows in reader]
    infile.close()
labels_2.remove(labels_2[0])
labels_2 = np.array(labels_2, dtype=np.uint8)



In [11]:
print(np.sum(labels_1))
print(np.sum(labels_2))

new_predictions = np.multiply(labels_1, labels_2)
print(np.sum(new_predictions))

1043
1211
634


In [14]:
df = pd.DataFrame(data = list(zip(filenames,list(new_predictions))))
df.to_csv("intersection.csv", index=False, header=["filename", "stalled"])

In [13]:
print(list(new_predictions))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 